In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load your trained YOLOv8 model
model = YOLO('best.pt')  # Replace with 'yolov8n.pt' if needed

# Load corner points calibration data
corner_points = np.load('corner_points.npz')['corner_points']
tl, tr, bl, br = corner_points[:4]

# Define class names (must match your trained model)
class_names = ['AA', 'D type', '9v']

# Initialize camera (adjust camera_index as needed)
camera_index = 1  # Example: change this to match your external camera index
cap = cv2.VideoCapture(camera_index)

if not cap.isOpened():
    print(f"Error: Could not open camera with index {camera_index}")
    exit()

# Function to map detected position to normalized grid coordinates
def map_to_grid(x, y, tl, tr, bl, br):
    # Compute vectors for mapping to normalized grid coordinates
    v1 = np.array([tr[0] - tl[0], tr[1] - tl[1]])
    v2 = np.array([bl[0] - tl[0], bl[1] - tl[1]])
    vp = np.array([x - tl[0], y - tl[1]])

    # Calculate normalized grid coordinates (u, v)
    u = np.dot(vp, v1) / np.dot(v1, v1)
    v = np.dot(vp, v2) / np.dot(v2, v2)

    return u, v

# Function to convert grid coordinates to robot coordinates
def grid_to_robot(u, v, tl, tr, bl, br):
    # Calculate the robot coordinates (X, Y, Z) from the normalized grid coordinates (u, v)
    x_robot = tl[0] + u * (tr[0] - tl[0]) + v * (bl[0] - tl[0])
    y_robot = tl[1] + u * (tr[1] - tl[1]) + v * (bl[1] - tl[1])
    z_robot = tl[2] + u * (tr[2] - tl[2]) + v * (bl[2] - tl[2])  # Adjust for height if necessary

    return x_robot, y_robot, z_robot

try:
    while True:
        ret, frame = cap.read()  # Capture frame-by-frame

        if not ret:
            print("Error: Failed to capture image")
            break

        # Perform inference
        results = model(frame)[0]  # Get the first result

        # Iterate through the detected objects
        detections = []
        for result in results.boxes.data:
            x1, y1, x2, y2, confidence, class_id = result.tolist()

            # Calculate center coordinates of the bounding box
            x_center = (x1 + x2) / 2
            y_center = (y1 + y2) / 2

            # Map the detected position to the grid coordinates
            u, v = map_to_grid(x_center, y_center, tl, tr, bl, br)

            # Convert grid coordinates to robot coordinates
            x_robot, y_robot, z_robot = grid_to_robot(u, v, tl, tr, bl, br)

            # Save detection information (battery type and coordinates)
            detections.append({
                'battery_type': class_names[int(class_id)],
                'robot_coordinates': (x_robot, y_robot, z_robot)
            })

            # Draw bounding box and label
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            label = f'{class_names[int(class_id)]} {confidence:.2f}'
            cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Display the frame with detections
        cv2.imshow('Battery Detection', frame)

        # Print or log the detected battery information
        for detection in detections:
            print(f"Detected {detection['battery_type']} at robot coordinates {detection['robot_coordinates']}")

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Detection stopped by user.")
finally:
    # Release the capture and close all windows
    cap.release()
    cv2.destroyAllWindows()


Error: Failed to capture image
